# Differentiable Histograms

One of the most common data structures in HEP is the **histogram**.

It is a very powerful data reduction and density estimation technique.

Unfortunately it is not differentiable :(. However, through the use
of "soft relaxations" we can achieve *differentiable histograms*.

Check out this preprint https://arxiv.org/abs/2203.05570 for more information, but at the core these
are binned Kernel Density Estimates (bKDE)

In [ ]:
import relaxed
import numpy as np
import matplotlib.pyplot as plt

The trick is that instead of using a "hard" histogram function like `np.histogram` (`jnp.histogram` doesn't mean it's differentiable, but for fun you can try what happens if you use that) you would use a "relaxed" version from the `relaxed` library.

As a user you will need to specify the degree of relaxation and this is given by the "bandwidth" of the kernel density estimate. Try playing with the bandwidth parameter to see the effect.

In [ ]:
inputs = np.random.normal(1, 1, size=1000)
bins = np.linspace(-10, 10, 51)
ctrs = bins[:-1] + np.diff(bins) / 2
counts_soft_p1 = relaxed.hist(inputs, bandwidth=0.5, bins=bins)
counts_soft_p2 = relaxed.hist(inputs, bandwidth=1.0, bins=bins)

counts_hard, _ = np.histogram(inputs, bins=bins)

In [ ]:
plt.step(ctrs, counts_hard, label="hard", where="mid")
plt.step(ctrs, counts_soft_p1, label="small bw", where="mid")
plt.step(ctrs, counts_soft_p2, label="large bw", where="mid")
plt.legend()
plt.xlim(-5, 5)

# Optimizing through histogramming


The main advantage if "soft" histograms is that you can optimize (or take gradients) throughout the histogramming operation. We will demonstrate this by doing a "gradient-based" shift of the histogram.

I.e. if you have many events $x_i$ what is the value $\Delta$ such that the peak of the histogram of  $x_i + \Delta$ is at some target value (e.g. t=3.0)

For this we will set up a loss term 

$$L(\Delta) = (\mathrm{mean}(\mathrm{histo}(x_i+\Delta)) - t)^2$$

and we will the minimize $L$ by computing gradients 

$$\partial L \over \partial \Delta$$


In [2]:
def shift_histo(factor, inputs):
    shifted_inputs = factor + inputs
    #     soft_counts,_ = jax.numpy.histogram(shifted_inputs, bins = bins)
    soft_counts = relaxed.hist(shifted_inputs, bandwidth=0.5, bins=bins)
    binned_mean = np.sum(ctrs * soft_counts / len(shifted_inputs))
    return binned_mean


def loss(param, target, inputs):
    binned_mean = shift_histo(param, inputs)
    loss = (binned_mean - target) ** 2
    return loss

In [ ]:
import jax
import optax

opt = optax.adam(4e-2)
params = 0.0
state = opt.init(params)
target = 3.0

for i in range(100):
    lvalue, g = jax.value_and_grad(loss)(params, target, inputs)
    updates, state = opt.update(g, state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        print(params, lvalue)

        hard_hist = np.histogram(params + inputs, bins=bins)[0]
        hard_mean = np.sum(ctrs * hard_hist / len(inputs))
        plt.step(ctrs, hard_hist, where="mid", color="maroon")
        plt.xlim(-5, 5)
        plt.ylim(0, 200)
        plt.vlines(hard_mean, 0, 200, colors="maroon")
        plt.vlines(target, 0, 200, colors="k")
        plt.show()